# Fermionic Dimer

In [ ]:
using Pkg; Pkg.activate()
using KadanoffBaym
using LinearAlgebra

In [ ]:
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")
using LaTeXStrings

## Model

### Tight-binding Hamiltonian:

\begin{align}\begin{split}
	\hat{H} &= \sum_{i=1}^L \varepsilon_i \hat{c}_i^{\dagger} \hat{c}_i^{\phantom{\dagger}} + J \sum_{\langle i, j\rangle}\left(\hat{c}_i^{\dagger} \hat{c}_j^{\phantom{\dagger}} + \hat{c}_j^{\dagger} \hat{c}_i^{\phantom{\dagger}}\right)
\end{split}\end{align}

### Green functions

\begin{align}\begin{split}
	\left[\boldsymbol{G}^>(t, t')\right]_{ij} &= G^>_{ij}(t, t') = -i\left\langle{\hat{c}_i^{\phantom{\dagger}}(t)\hat{c}_j^{{\dagger}}(t')}\right\rangle \\
	\left[\boldsymbol{G}^<(t, t')\right]_{ij} &= G^<_{ij}(t, t') = \phantom{-} i\left\langle{\hat{c}_j^{{\dagger}}(t')\hat{c}_i^{\phantom{\dagger}}(t)}\right\rangle
\end{split}\end{align}

### Equations of motion

\begin{align}\begin{split}
	i\partial_t \boldsymbol{G}^{\lessgtr}(t, t') &= \boldsymbol{H} \boldsymbol{G}^{\lessgtr}(t, t') \\
	i\partial_T \boldsymbol{G}^{\lessgtr}(T, 0)_W &= [\boldsymbol{H},\boldsymbol{G}^{\lessgtr}(T, 0)_W]
\end{split}\end{align}

\begin{align}\begin{split}
	\boldsymbol{H} &= 
	\begin{pmatrix}
		\varepsilon_1 & J      &        &   \\
		J             & \ddots & \ddots &   \\
		              & \ddots & \ddots & J \\
		              &        & J & \varepsilon_L 
	\end{pmatrix}
\end{split}\end{align}

## Solving the dimer model

In [ ]:
# final time
T = 5.0

# Hamiltonian matrix
ε₁ = 1.0
ε₂ = -1.0
J = 20.0
H = ComplexF64[ε₁ J; J ε₂];

In [ ]:
# quantum numbers
dim = 2

# Allocate the initial Green functions (time arguments at the end)
GL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
GG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)

# initial condition
N_0 = 1.0

GL[1, 1, 1, 1] = 1.0im * N_0
GG[:, :, 1, 1] = -1.0im .* diagm([1.0, 1.0]) .+ GL[:, :, 1, 1]

# right-hand side for the "vertical" evolution
function fv!(out, times, t1, t2)
    out[1] = -1.0im * H * GL[t1, t2]
    out[2] = -1.0im * H * GG[t1, t2]
end

# right-hand side for the "diagonal" evolution
function fd!(out, times, t1, t2)
  fv!(out, times, t1, t2)
  out[1] .-= adjoint(out[1])
  out[2] .-= adjoint(out[2])
end

# Analytic result
ana(t1, t2) = exp(-1.0im * H * t1) * GL[:, :, 1, 1] * exp(1.0im * H * t2);

In [ ]:
# call the solver
sol = kbsolve!(fv!, fd!, [GL, GG], (0.0, T); atol=1e-9, rtol=1e-7);

## Example plots

In [ ]:
# two quantum numbers to plot
idx1 = 1
idx2 = 1;

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)
plot(J .* sol.t, [imag(GL.data[idx1, idx2, k, k]) for k in 1:length(sol.t)], marker="", ms=3.0, ls="-", c="C0")
ax.set_xlim(0, J * 1)
ax.set_ylim(0, N_0)
ax.set_xticks(J .* [0, 0.5, 1])
ax.set_yticks([0, 0.5, 1])
ax.set_xlabel(L"J t")
ax.set_ylabel(L"\mathrm{Im}G^<_{11}(t, t)")
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_axisbelow(false)
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

ax = subplot(122)
plot(J .* sol.t, [imag(GL.data[idx1, idx2, k, k] - ana(sol.t[k], sol.t[k])[idx1, idx2]) for k in eachindex(sol.t)], marker="", ms=3.0, ls="-", c="r")
ax.set_xlim(0, J * 5)
ax.set_ylim((-5, 5) .* 1e-4)
ax.set_xticks(J .* [0, 1, 2, 3, 4, 5])
ax.set_xlabel(L"J t")
ax.set_ylabel(L"\mathrm{Im}\left[G^<_{11}(t, t) - \mathcal{G}^<_{11}(t, t)\right]", labelpad=16)
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.yaxis.set_label_position("right")
ax.set_axisbelow(false)
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.1, w_pad=0.75, h_pad=0)
# savefig("fermion_example_1.pdf")

## Adaptive vs fixed step size

In [ ]:
# short final time
T = 0.02;

sol_adaptive = kbsolve!(fv!, fd!, [GL, GG], (0.0, T); dtini=1e-8, atol=1e-14, rtol=1e-5, γ=99/100);
dts_adaptive = sol_adaptive.t[2:end] .- sol_adaptive.t[1:end-1];
@show size(dts_adaptive)
@show error = let
  s = [ana(t1, t2)[1, 1] for t1 in sol_adaptive.t, t2 in sol_adaptive.t]
  e = abs.((GL[1,1,:,:] - s) ./ s)
  sum(e) / *(size(e)...)
end


sol_fixed = kbsolve!(fv!, fd!, [GL, GG], (0.0, T); dtini=1e-8, rtol=1e-5, dtmax=5e-4, atol=1e-14, γ=1)
dts_fixed = sol_fixed.t[2:end] .- sol_fixed.t[1:end-1];
@show size(dts_fixed)
@show error = let
  s = [ana(t1, t2)[1, 1] for t1 in sol_fixed.t, t2 in sol_fixed.t]
  e = abs.((GL[1,1,:,:] - s) ./ s)
  sum(e) / *(size(e)...)
end;

sol_fixed2 = kbsolve!(fv!, fd!, [GL, GG], (0.0, T); dtini=5e-4, rtol=1e0, dtmax=5e-4, atol=1e-14, γ=1)
dts_fixed2 = sol_fixed2.t[2:end] .- sol_fixed2.t[1:end-1];
@show size(dts_fixed2)
@show error = let
  s = [ana(t1, t2)[1, 1] for t1 in sol_fixed2.t, t2 in sol_fixed2.t]
  e = abs.((GL[1,1,:,:] - s) ./ s)
  sum(e) / *(size(e)...)
end;

In [ ]:
fig = figure(figsize=(3.5, 3))
ax = fig.add_subplot(111)

# we discard the points that are outside of ylim (since we are drawing on top of the canvas)
ax.semilogy(J .* sol_adaptive.t[10:end], dts_adaptive[9:end], "s-C0", lw=1.5, ms=5, label="adaptive", markeredgecolor="#22577c", clip_on=false, zorder=100)
ax.semilogy(J .* sol_fixed.t[10:end], dts_fixed[9:end], "o-C3", lw=1, ms=4, label="semi-fixed", alpha=0.9, clip_on=false, zorder=100)
ax.semilogy(J .* sol_fixed2.t[2:end], dts_fixed2, "o-k", lw=1, ms=2, label="fixed", zorder=100)

ax.tick_params(axis="y", which="minor")
xlim(0-3e-4, T+3e-4)
xlim(0, J * T)
ylim(1e-5, 1e-2)

xlabel(L"J t")
ylabel(L"h")
legend(loc="lower right", frameon=false, labelspacing=0.2, borderpad=0.0, handlelength=1.5, fontsize=15)
tight_layout(pad=0.3)
# savefig("fermion_example_adaptive_dt.pdf")

## Error scaling

In [ ]:
using LsqFit

In [ ]:
epsilons = [10^(-k) for k in range(3, 10; length=34)]

err_data = []
p_norm = 1

for (k, eps) in enumerate(epsilons)
    print("$k, ")
    
    sol = kbsolve!(fv!, fd!, [GL, GG], (0.0, 5.0); dtini=1e-10, atol=1e-2eps, rtol=eps, kmax=9);
    
    err = norm(
    [GL.data[idx1, idx2, t1, t2] - ana(sol.t[t1], sol.t[t2])[idx1, idx2] for t1 in eachindex(sol.t), t2 in eachindex(sol.t)], p_norm)
    
    push!(err_data, (length(sol.t), err / length(sol.t)^2, eps))
end

In [ ]:
xdata = log10.([x[1] for x in err_data])
ydata = log10.([x[2] for x in err_data]);

In [ ]:
fit_func = (n, p) -> -p[1] .* n .+ p[2];
fit_result = curve_fit(fit_func, xdata, ydata, [2.0, 1]);
coef(fit_result)

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
plot(xdata, map(x -> fit_func(x, coef(fit_result)), xdata), "--k", lw=2,
     label=L"\mathcal{O}(h^{%$(floor(coef(fit_result)[1], sigdigits=4))})")
plot(xdata, ydata, "o", lw=0, ms=6,
     markerfacecolor="C0", markeredgewidth=0.25, markeredgecolor="#2D5FAA")
ax.set_xlabel(L"\log(n)")
ax.set_ylabel(L"\log(\epsilon_{11})")
legend(loc="best", handlelength=1.8, frameon=false, borderpad=0, labelspacing=0)

ax = subplot(122)
plot(log10.([x[3] for x in err_data]), log10.([x[1] for x in err_data]), "o", ms=5,
     markerfacecolor="C0", markeredgewidth=0.25, markeredgecolor="#2D5FAA")
ax.set_xlim(-2, -10.3)
ax.set_xticks([-2, -4, -6, -8, -10])
ax.yaxis.set_label_position("right")
ax.set_xlabel(L"\log(\texttt{rtol})")
ax.set_ylabel(L"\log(n)", labelpad=16)

tight_layout(pad=0.1, w_pad=0.75, h_pad=0)
# savefig("fermion_example_error_scaling.pdf")